In [16]:
import pandas as pd
from candecoder import decode_message_by_id

In [17]:

# opening and reading data from records.csv file
data = pd.read_csv('./records.csv')

In [18]:

# creates column titles
header = ['Timestamp', 'Variable', 'Value', 'Units']

# replaces square brackets and single quotes for correct formatting and type conversions

processed_data = pd.DataFrame(columns=header)

required_cols = ["rawtime"] + ["id"] + [f"data{n}" for n in range(0, 8)]

In [19]:
from datetime import timedelta, datetime


def extractDateInfo(rawtime):
    days = timedelta(days=rawtime)
    date = datetime(year=1899, month=12, day=30) + days
    return pd.to_datetime(date)

In [20]:
from ast import literal_eval
def map_row(rawtime, id, *hex_bytes):
    try:
        data_bytes = [int(hex_byte, 16) for hex_byte in hex_bytes]
        values = decode_message_by_id(id, data_bytes)
        ret_values = pd.DataFrame(columns=header)
        for variable, value in values.items():
            if type(value) == tuple:
                ret_values.loc[len(ret_values)] = {
                    "Timestamp": extractDateInfo(rawtime),
                    "Variable": variable,
                    "Value": value[0],
                    "Units": value[1]
                }
            else:
                ret_values.loc[len(ret_values)] = {
                    "Timestamp": extractDateInfo(rawtime),
                    "Variable": variable,
                    "Value": value,
                    "Units": None
                }
        return ret_values
    except:
        return None

In [21]:
data_cut = data.iloc[:1000]

In [22]:
#from tqdm.notebook import tqdm

#dfs = [map_row(*row) for row in tqdm(zip(*[data_cut[col] for col in required_cols]))]
#dfs = [map_row(*row) for row in [data_cut[col] for col in required_cols] ]

#data_cut.head()
dfs = [map_row(*row) for row in zip(*[data_cut[col] for col in required_cols]) ]

# if packet is not None for row in map_row(*[packet[field] for field in range(len(required_cols))])]

In [23]:
dfs = pd.concat(dfs)

In [25]:
#dfs.to_csv("./processed_records.csv")
dfs

,Timestamp,Variable,Value,Units
0,2023-02-27 18:24:36.203302,Inverter2_Modulation_Index_&_Flux_Weakening_Ou...,0.02,None
1,2023-02-27 18:24:36.203302,Inverter2_Modulation_Index_&_Flux_Weakening_Ou...,0.00,amps
2,2023-02-27 18:24:36.203302,Inverter2_Modulation_Index_&_Flux_Weakening_Ou...,0.00,0
3,2023-02-27 18:24:36.203302,Inverter2_Modulation_Index_&_Flux_Weakening_Ou...,0.00,None
0,2023-02-27 18:24:36.203502,Inverter2_Torque_&_Timer_Information_Commanded...,0.00,N-m
...,...,...,...,...
52,2023-02-27 18:24:36.882902,Inverter1_Fault_Codes_Current Sensor Fault,0.00,None
53,2023-02-27 18:24:36.882902,Inverter1_Fault_Codes_Gen 3: Reserved / Gen 5:...,0.00,None
54,2023-02-27 18:24:36.882902,Inverter1_Fault_Codes_Gen 3: Hardware DC Bus O...,0.00,None
55,2023-02-27 18:24:36.882902,Inverter1_Fault_Codes_Gen 3: Reserved / Gen 5:...,0.00,None
